In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import pickle
import shelve
import klepto
from gensim import corpora, models
import feather
from collections import defaultdict
import itertools

In [2]:
poor_data = pd.read_pickle("/Users/Hun/hse_thesis/hse_thesis1.pickle")

In [3]:
add_data = pd.read_csv("/Users/Hun/hse_thesis/metadata.csv", encoding="CP1251",
                        error_bad_lines=False, warn_bad_lines=False,
                       header=None)

/Users/Hun/Python_env/Python3_env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
add_data.drop(0, axis=1, inplace=True)

In [5]:
col_names=["author_id", "wall_id", "post_comment_id", "date", "name", "surname", "sex", "city", "region",
      "country", "posts", "comments", "likes", "reposts", "res_postlikes", "res_comments", "university",
       "freinds", "groups", "followers"]

In [6]:
len(col_names)

20

In [7]:
add_data.columns = col_names

In [8]:
add_data["wall_id"] = add_data["wall_id"].apply(lambda text: int(text.split()[-1]))

add_data["post_comment_id_int"] = add_data["post_comment_id"].apply(lambda text: int(text.split()[-1]))
add_data["is_post"] = add_data["post_comment_id"].apply(lambda text: text.split()[0] == "post")
                                                        
poor_data["post_comment_id_int"] = poor_data["post_comment_id"].apply(lambda text: int(text.split()[-1]))
poor_data["is_post"] = poor_data["post_comment_id"].apply(lambda text: text.split()[0] == "post")
                                                        
add_data.drop(["post_comment_id", "date", "name", "surname", "city", "region", "country"], axis=1, inplace=True)
poor_data.drop("post_comment_id", axis=1, inplace=True)

In [9]:
add_data.head()

,author_id,wall_id,sex,posts,comments,likes,reposts,res_postlikes,res_comments,university,freinds,groups,followers,post_comment_id_int,is_post
0,85978354,112465206,2,1.0,0.0,0.0,0.0,0.0,0.0,,36.0,789.0,272.0,1,True
1,28098487,28098487,2,12.0,0.0,2.0,0.0,18.0,0.0,,59.0,55.0,8.0,249,True
2,28098487,28098487,2,12.0,0.0,2.0,0.0,18.0,0.0,,59.0,55.0,8.0,246,True
3,28098487,28098487,2,12.0,0.0,2.0,0.0,18.0,0.0,,59.0,55.0,8.0,245,True
4,28098487,28098487,2,12.0,0.0,2.0,0.0,18.0,0.0,,59.0,55.0,8.0,244,True


In [10]:
add_data.isnull().sum()

author_id                   0
wall_id                     0
sex                    114341
posts                  114341
comments               114341
likes                  114341
reposts                114341
res_postlikes          114341
res_comments           114341
university             114341
freinds                226633
groups                 226633
followers              226633
post_comment_id_int         0
is_post                     0
dtype: int64

In [11]:
add_data.shape

(8166504, 15)

In [12]:
add_data.drop_duplicates(subset=["wall_id", "post_comment_id_int", "author_id", "is_post"], inplace=True)

In [13]:
add_data.shape

(8079280, 15)

In [14]:
add_data.isnull().sum()

author_id                   0
wall_id                     0
sex                     54613
posts                   54613
comments                54613
likes                   54613
reposts                 54613
res_postlikes           54613
res_comments            54613
university              54613
freinds                166892
groups                 166892
followers              166892
post_comment_id_int         0
is_post                     0
dtype: int64

In [15]:
data = pd.merge(poor_data, add_data, on=["wall_id", "post_comment_id_int", "author_id", "is_post"], how="left")

In [16]:
data.head()

,text,author_id,wall_id,date,name,surname,city,region,text_length,post_comment_id_int,...,posts,comments,likes,reposts,res_postlikes,res_comments,university,freinds,groups,followers
0,general lee,49154141,49154141,2012-02-24 11:54:20,Дмитрий,Филиппов,Владимир,Владимирская область,2,2404,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0
1,shelby,49154141,49154141,2012-02-24 11:54:33,Дмитрий,Филиппов,Владимир,Владимирская область,1,2405,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0
2,ford mustang,49154141,49154141,2012-02-25 22:27:35,Дмитрий,Филиппов,Владимир,Владимирская область,2,2411,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0
3,murcielago,49154141,49154141,2012-02-24 11:53:51,Дмитрий,Филиппов,Владимир,Владимирская область,1,2400,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0
4,суровый водитель,49154141,49154141,2012-02-23 16:22:56,Дмитрий,Филиппов,Владимир,Владимирская область,2,2392,...,2133.0,0.0,1352.0,0.0,2670.0,0.0,,198.0,106.0,113.0


In [17]:
poor_data.shape, add_data.shape, data.shape

((5392586, 11), (8079280, 15), (5392586, 22))

In [18]:
data.isnull().sum()

text                        0
author_id                   0
wall_id                     0
date                        0
name                    48759
surname                 49346
city                   411426
region                 421852
text_length                 0
post_comment_id_int         0
is_post                     0
sex                    184556
posts                  184556
comments               184556
likes                  184556
reposts                184556
res_postlikes          184556
res_comments           184556
university             184556
freinds                263451
groups                 263451
followers              263451
dtype: int64

In [ ]:
data.to_pickle("/Users/Hun/hse_thesis/hse_thesis1.pickle")

Ограничиваем данные постами, написанными позже 20 октября 2010 года и считаем их количество: